MAPLES-DR Intervariability Study
================================

In [1]:
# Import maples-dr
import maples_dr
import numpy as np
import pandas as pd

# Import visualization tools
from ipywidgets import HTML, Dropdown, GridBox, Layout
from jppype import imshow, sync_views, vscode_theme
from jppype.utilities.geometric import Point
from maples_dr.dataset import BiomarkerField as Biomarkers
from maples_dr.dataset import FundusField as Fundus
from maples_dr.quick_api import GLOBAL_LOADER
from sklearn.metrics import cohen_kappa_score

# Import utilities
from variability_study_utils import centroid, connected_components_f1

vscode_theme()

Thanks for using MAPLES-DR!
  When using this dataset in academic works,
  please cite: ]8;id=807725;https://arxiv.org/abs/2402.04258\https://arxiv.org/abs/2402.04258]8;;\

HTML(value='<style>\n    .cell-output-ipywidget-background {\n    background-color: transparent !important;\n …

In [2]:
maples_dr.configure(
    maples_dr_path="PATH/TO/MAPLES-DR/MAPLES-DR_v2.zip",
    messidor_path="PATH/TO/MESSIDOR/",
    image_format="bgr",
    preprocessing="clahe",
)
maples_dataset = GLOBAL_LOADER.load_dataset("all_with_duplicates")

## Variability study on the duplicated images

In [3]:
duplicates = list(GLOBAL_LOADER.maples_dr_infos["duplicates"].items())

#### Qualitative study

In [4]:
ID = 0
sample1, sample2 = [maples_dataset[_] for _ in duplicates[ID]]

In [5]:
selectors = [
    Dropdown(
        options=[field.value for field in Biomarkers],
        description="Biomarker:",
        layout=Layout(width="auto"),
        value=("brightLesions", "redLesions", "vessels")[i],
    )
    for i in range(3)
]
views = [imshow(sample1["fundus"]) for i in range(3)]

for i in range(3):

    def set_label(biomarker, i=i):
        if isinstance(biomarker, dict):
            biomarker = biomarker["new"]
        views[i].add_label(
            sample1[biomarker] + 2 * sample2[biomarker],
            colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
            name="biomarker",
        )

    selectors[i].observe(set_label, "value")

    set_label(selectors[i].value)

sync_views(*views)

GridBox(
    selectors + views,
    layout=Layout(grid_template_columns="repeat(3, 1fr)", height="600px"),
)

GridBox(children=(Dropdown(description='Biomarker:', index=4, layout=Layout(width='auto'), options=('opticCup'…

In [6]:
selector = Dropdown(
    options=[field.value for field in Biomarkers],
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=("brightLesions", "redLesions", "vessels")[i],
)

duplicates_samples = [[maples_dataset[_[0]], maples_dataset[_[1]]] for _ in duplicates]
(s1a, s1b), (s2a, s2b) = duplicates_samples
view1 = imshow(s1a["fundus"])
view2 = imshow(s1a["fundus"])


def set_label(biomarker):
    if isinstance(biomarker, dict):
        biomarker = biomarker["new"]
    view1.add_label(
        s1a[biomarker] + 2 * s1b[biomarker],
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )
    view2.add_label(
        s1a.read_biomarker(biomarker, pre_annotation=True) + 2 * s1b.read_biomarker(biomarker, pre_annotation=True),
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )


selector.observe(set_label, "value")

set_label(selectors[i].value)

sync_views(view1, view2)

GridBox(
    [selector, HTML(), view1, view2],
    layout=Layout(grid_template_columns="repeat(2, 1fr)", grid_template_rows="auto 600px"),
)

GridBox(children=(Dropdown(description='Biomarker:', index=3, layout=Layout(width='auto'), options=('opticCup'…

In [7]:
selector = Dropdown(
    options=[field.value for field in Biomarkers],
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=("brightLesions", "redLesions", "vessels")[i],
)

duplicates_samples = [[maples_dataset[_[0]], maples_dataset[_[1]]] for _ in duplicates]
(s1a, s1b), (s2a, s2b) = duplicates_samples
view1b = imshow(s2a["fundus"])
view2b = imshow(s2a["fundus"])


def set_label(biomarker):
    if isinstance(biomarker, dict):
        biomarker = biomarker["new"]
    view1b.add_label(
        s2a[biomarker] + 2 * s2b[biomarker],
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )
    view2b.add_label(
        s2a.read_biomarker(biomarker, pre_annotation=True) + 2 * s2b.read_biomarker(biomarker, pre_annotation=True),
        colormap={1: "#a56fb9", 2: "#7aa8ba", 3: "white"},
        name="biomarker",
    )


selector.observe(set_label, "value")

set_label(selectors[i].value)

sync_views(view1b, view2b)

GridBox(
    [selector, HTML(), view1b, view2b],
    layout=Layout(grid_template_columns="repeat(2, 1fr)", grid_template_rows="auto 600px"),
)

GridBox(children=(Dropdown(description='Biomarker:', index=3, layout=Layout(width='auto'), options=('opticCup'…

### Quantitative

In [8]:
biomarkers = [
    field.value
    for field in (
        Biomarkers.VESSELS,
        Biomarkers.OPTIC_CUP,
        Biomarkers.OPTIC_DISC,
        Biomarkers.MACULA,
        Biomarkers.RED_LESIONS,
        Biomarkers.BRIGHT_LESIONS,
    )
]

data = {}
for bio in biomarkers:
    sample_a = np.array([s1a[bio], s2a[bio]], dtype=bool)
    sample_b = np.array([s1b[bio], s2b[bio]], dtype=bool)
    kappa = cohen_kappa_score(sample_a.flatten(), sample_b.flatten(), labels=[0, 1])
    accuracy = np.mean(sample_a == sample_b)
    dice = 2 * np.sum(sample_a * sample_b) / (np.sum(sample_a) + np.sum(sample_b))
    data[bio] = {"kappa": kappa, "accuracy": accuracy, "dice": dice}

for bio in [Biomarkers.MACULA, Biomarkers.OPTIC_CUP, Biomarkers.OPTIC_DISC]:
    d = np.mean([centroid(sa[bio]).distance(centroid(sb[bio])) for sa, sb in duplicates_samples])
    data[bio.value]["distance"] = d

for bio in [Biomarkers.RED_LESIONS, Biomarkers.BRIGHT_LESIONS]:
    data[bio.value]["mean detection f1"] = np.nanmean(
        [connected_components_f1(sa[bio], sb[bio]) for sa, sb in duplicates_samples]
    )


pd.DataFrame(data).round(3).fillna("")

/home/gaby/These/src/Fundus/MAPLES-DR/examples/variability_study_utils.py:28: RuntimeWarning: invalid value encountered in divide
  spe /= nb_l2


,vessels,opticCup,opticDisc,macula,redLesions,brightLesions
kappa,0.802,0.847,0.958,0.045,0.48,0.176
accuracy,0.961,0.999,0.999,0.999,0.999,0.998
dice,0.824,0.848,0.958,0.045,0.48,0.177
distance,,5.581,2.365,24.316,,
mean detection f1,,,,,0.616,0.083


In [9]:
from skimage.measure import label

np.max(label(s2a[Biomarkers.RED_LESIONS])), np.max(label(s2b[Biomarkers.RED_LESIONS]))

(9, 13)

In [10]:
n = 0
for s in maples_dataset:
    n += s[Biomarkers.NEOVASCULARIZATION].any()

6